<a href="https://colab.research.google.com/github/MohamadRayi/MachineLearningTasks/blob/main/Task%20Lecture%2013%20Machine%20Learning/BAB%203/A_full_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mohamad Rayi Dwi Putra

1103213070

Task Lecture 13

A_full_training

Beberapa kodingan Tidak ditampilkan karena memerlukan waktu yang lama untuk menampilkan output nya

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 1.2 MB/s eta 

In [2]:
# Mengimpor fungsi load_dataset dari pustaka datasets
from datasets import load_dataset

# Mengimpor kelas AutoTokenizer dan DataCollatorWithPadding dari pustaka Transformers
from transformers import AutoTokenizer, DataCollatorWithPadding

# Memuat dataset GLUE (General Language Understanding Evaluation) dengan tugas MRPC (Microsoft Research Paraphrase Corpus)
# Dataset ini digunakan untuk mengevaluasi kemampuan model dalam memahami dan memprediksi apakah dua kalimat adalah parafrasa
raw_datasets = load_dataset("glue", "mrpc")

# Menentukan model yang akan digunakan dengan checkpoint bert-base-uncased
# Model ini adalah versi uncased dari BERT base, yang berarti semua teks diubah menjadi huruf kecil
checkpoint = "bert-base-uncased"

# Membuat objek tokenizer otomatis berdasarkan model yang dipilih
# Tokenizer ini akan digunakan untuk memproses teks menjadi token yang dapat dimengerti oleh model BERT
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Membuat fungsi untuk melakukan tokenisasi pada dataset
# Fungsi ini mengambil dua kalimat (sentence1 dan sentence2) dan mengembalikan token yang dipotong sesuai kebutuhan (truncation=True)
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Melakukan tokenisasi pada dataset menggunakan fungsi yang telah didefinisikan sebelumnya, dengan batch processing
# Fungsi map dengan parameter batched=True akan mengaplikasikan fungsi tokenisasi pada setiap batch dalam dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Membuat objek data_collator untuk menangani padding pada data dengan tokenizer yang telah digunakan
# DataCollatorWithPadding akan memastikan semua urutan dalam batch memiliki panjang yang sama dengan menambahkan padding jika diperlukan
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
# Menghapus kolom "sentence1", "sentence2", dan "idx" dari dataset tokenized_datasets
# Kolom-kolom ini tidak diperlukan lagi setelah tokenisasi dan dapat dihapus untuk menghemat memori
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

# Mengubah nama kolom "label" menjadi "labels" dalam dataset tokenized_datasets
# Model BERT biasanya mengharapkan kolom target diberi nama "labels"
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Mengatur format dataset tokenized_datasets menjadi "torch"
# Format ini memungkinkan penggunaan langsung dengan model PyTorch, mengubah semua fitur menjadi tensor PyTorch
tokenized_datasets.set_format("torch")

# Mendapatkan nama kolom dari bagian "train" dalam dataset tokenized_datasets
# Ini untuk memastikan kolom yang diperlukan sudah sesuai dan memverifikasi perubahan yang dilakukan sebelumnya
tokenized_datasets["train"].column_names


['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [4]:
["attention_mask", "input_ids", "labels", "token_type_ids"]

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [5]:
# Mengimpor kelas DataLoader dari modul torch.utils.data
from torch.utils.data import DataLoader

# Membuat DataLoader untuk data latih (train)
# tokenized_datasets["train"]: dataset yang sudah di-tokenisasi untuk data latih
# shuffle=True: data akan diacak setiap epoch
# batch_size=8: jumlah sampel yang akan dimuat dalam setiap batch
# collate_fn=data_collator: fungsi untuk menggabungkan sampel-sampel ke dalam batch
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

# Membuat DataLoader untuk data evaluasi (validasi)
# tokenized_datasets["validation"]: dataset yang sudah di-tokenisasi untuk data evaluasi
# batch_size=8: jumlah sampel yang akan dimuat dalam setiap batch
# collate_fn=data_collator: fungsi untuk menggabungkan sampel-sampel ke dalam batch
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [6]:
# Mengulang setiap batch dalam dataloader pelatihan (train_dataloader)
for batch in train_dataloader:
    # Menghentikan iterasi setelah satu batch (ini mungkin hanya contoh atau debugging)
    break

# Membuat kamus yang berisi bentuk (shape) dari setiap tensor dalam batch
# 'k' adalah kunci (key), mewakili nama tensor, dan 'v' adalah nilai (value), mewakili bentuk tensor
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 70]),
 'token_type_ids': torch.Size([8, 70]),
 'attention_mask': torch.Size([8, 70])}

In [7]:
# Mengimpor kelas yang diperlukan dari pustaka Transformers untuk model klasifikasi urutan otomatis
from transformers import AutoModelForSequenceClassification

# Membuat objek model klasifikasi urutan otomatis dan memuat berat dari titik kontrol yang disebutkan dengan menentukan jumlah label
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Melakukan pemrosesan forward pass model dengan batch data yang diberikan
outputs = model(**batch)

# Mencetak nilai loss dari output model serta bentuk logits yang dihasilkan
print(outputs.loss, outputs.logits.shape)

tensor(0.5930, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [9]:
# Mengimpor kelas optimizer AdamW dari pustaka Transformers
from transformers import AdamW

# Membuat objek optimizer AdamW dengan model.parameters() sebagai parameter untuk dioptimalkan
# dan learning rate (lr) sebesar 5e-5
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
# Mengimpor fungsi get_scheduler dari pustaka Transformers
from transformers import get_scheduler

# Menentukan jumlah epochs yang akan dilatih
num_epochs = 3

# Menghitung total langkah pelatihan berdasarkan jumlah epochs dan panjang dataloader pelatihan
# total langkah pelatihan = jumlah epochs * jumlah batch dalam train_dataloader
num_training_steps = num_epochs * len(train_dataloader)

# Membuat penjadwalan laju pembelajaran menggunakan metode linear
# Scheduler ini akan secara bertahap menyesuaikan laju pembelajaran dari optimizer selama pelatihan
# dengan menentukan optimizer yang digunakan, jumlah langkah pemanasan (warmup), dan total langkah pelatihan
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Mencetak total langkah pelatihan
# Ini untuk memverifikasi jumlah total langkah pelatihan yang dihitung berdasarkan epoch dan panjang dataloader
print(num_training_steps)


1377


In [11]:
# Mengimpor pustaka PyTorch
import torch

# Memeriksa ketersediaan GPU, jika tersedia, gunakan GPU, jika tidak, gunakan CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Memindahkan model ke perangkat yang telah ditentukan (GPU atau CPU)
model.to(device)

# Menampilkan perangkat yang digunakan (GPU atau CPU)
device

device(type='cpu')

In [ ]:
# Mengimpor kelas tqdm dari pustaka tqdm untuk menampilkan bar kemajuan
from tqdm.auto import tqdm

# Membuat objek progress_bar dengan rentang num_training_steps untuk menampilkan bar kemajuan saat pelatihan model
# tqdm memberikan visualisasi bar kemajuan untuk melacak proses pelatihan
progress_bar = tqdm(range(num_training_steps))

# Mengatur model dalam mode pelatihan
# model.train() memberitahu model bahwa sedang dalam fase pelatihan, mengaktifkan dropout dan batch normalization jika ada
model.train()

# Melakukan iterasi sebanyak num_epochs untuk melatih model
for epoch in range(num_epochs):
    # Melakukan iterasi untuk setiap batch dalam train_dataloader
    for batch in train_dataloader:
        # Memindahkan batch ke perangkat yang sesuai (misalnya, GPU)
        # Ini memastikan bahwa data diproses oleh perangkat yang digunakan untuk pelatihan (CPU atau GPU)
        batch = {k: v.to(device) for k, v in batch.items()}

        # Memperoleh output dari model berdasarkan batch input
        # Forward pass: model memproses input dan menghasilkan output
        outputs = model(**batch)

        # Mendapatkan nilai loss dari output model
        # Loss mengukur seberapa baik atau buruk prediksi model dibandingkan dengan label yang benar
        loss = outputs.loss

        # Menghitung gradien loss
        # Backward pass: menghitung gradien dari loss sehubungan dengan parameter model
        loss.backward()

        # Mengupdate parameter model berdasarkan gradien
        # Optimizer mengupdate parameter model untuk meminimalkan loss
        optimizer.step()

        # Mengupdate learning rate scheduler
        # Scheduler mengatur ulang learning rate sesuai jadwal yang ditentukan (misalnya, linear decay)
        lr_scheduler.step()

        # Mengatur gradien menjadi nol untuk iterasi selanjutnya
        # Mengatur gradien menjadi nol agar gradien dari iterasi sebelumnya tidak mempengaruhi iterasi berikutnya
        optimizer.zero_grad()

        # Mengupdate bar kemajuan
        # Menampilkan kemajuan pelatihan dengan bar kemajuan
        progress_bar.update(1)


In [13]:
# Mengimpor fungsi evaluate dari modul evaluate
import evaluate

# Memuat metrik evaluasi untuk dataset GLUE dan task MRPC
metric = evaluate.load("glue", "mrpc")

# Menetapkan mode evaluasi pada model
model.eval()

# Iterasi melalui dataloader untuk data evaluasi
for batch in eval_dataloader:
    # Memindahkan batch ke perangkat (device) yang sesuai (misalnya, GPU)
    batch = {k: v.to(device) for k, v in batch.items()}

    # Mematikan perhitungan gradien saat melakukan evaluasi
    with torch.no_grad():
        # Memperoleh output model menggunakan batch input
        outputs = model(**batch)

    # Mendapatkan prediksi dari logits menggunakan argumen maksimum
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    # Menambahkan batch prediksi dan label referensi ke metrik
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Menghitung dan mengembalikan nilai metrik
metric.compute()

{'accuracy': 0.8161764705882353, 'f1': 0.8704663212435233}

In [ ]:
# Mengimpor kelas-kelas yang diperlukan dari pustaka Transformers
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

# Menginisialisasi model berdasarkan checkpoint yang telah disediakan
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Menginisialisasi optimizer AdamW dengan learning rate 3e-5
optimizer = AdamW(model.parameters(), lr=3e-5)

# Memilih device untuk training (cuda jika tersedia, jika tidak, cpu)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)  # Memindahkan model ke device yang dipilih

# Mengatur jumlah epoch dan jumlah total langkah training
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

# Mengatur scheduler learning rate dengan metode linear
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Menampilkan progress bar menggunakan tqdm
progress_bar = tqdm(range(num_training_steps))

# Memasuki fase training
model.train()
for epoch in range(num_epochs):
    # Looping melalui data dalam dataloader
    for batch in train_dataloader:
        # Memindahkan batch ke device yang dipilih
        batch = {k: v.to(device) for k, v in batch.items()}
        # Melakukan forward pass
        outputs = model(**batch)
        # Menghitung loss
        loss = outputs.loss
        # Melakukan backpropagation
        loss.backward()
        # Melakukan update parameter dengan optimizer
        optimizer.step()
        # Melakukan update learning rate dengan scheduler
        lr_scheduler.step()
        # Mengosongkan gradient
        optimizer.zero_grad()
        # Memperbarui progress bar
        progress_bar.update(1)

In [ ]:
# Mengimpor kelas-kelas yang diperlukan dari pustaka Accelerate dan Transformers
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

# Inisialisasi objek Accelerator
# Accelerator mengatur distribusi pelatihan model ke beberapa GPU, TPU, atau CPU secara otomatis
accelerator = Accelerator()

# Mendapatkan model dari checkpoint yang sudah ada untuk klasifikasi urutan (sequence classification)
# Model ini digunakan untuk tugas klasifikasi dengan dua label (num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Mendefinisikan optimizer dengan AdamW dan learning rate 3e-5
# AdamW adalah varian dari Adam optimizer yang menggabungkan regularisasi L2 weight decay
optimizer = AdamW(model.parameters(), lr=3e-5)

# Mempersiapkan data training dan evaluasi menggunakan Accelerator
# prepare() memindahkan model, optimizer, dan dataloader ke perangkat yang sesuai (GPU/TPU/CPU) dan mengatur distribusi pelatihan
train_dl, eval_dl, model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader, model, optimizer)

# Mendefinisikan jumlah epoch dan jumlah langkah training
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)

# Mendefinisikan scheduler untuk learning rate
# Scheduler akan menyesuaikan learning rate secara bertahap selama pelatihan
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Inisialisasi progress bar menggunakan tqdm
# Progress bar akan menunjukkan kemajuan pelatihan
progress_bar = tqdm(range(num_training_steps))

# Mengatur model dalam mode pelatihan
# model.train() memberitahu model bahwa sedang dalam fase pelatihan, mengaktifkan dropout dan batch normalization jika ada
model.train()

# Melakukan iterasi untuk setiap epoch
for epoch in range(num_epochs):
    # Melakukan iterasi untuk setiap batch dalam data training
    for batch in train_dl:
        # Melakukan forward pass pada model
        # Model menerima batch input dan menghasilkan output
        outputs = model(**batch)
        # Mendapatkan loss dari output
        loss = outputs.loss
        # Melakukan backpropagation menggunakan Accelerator
        # Accelerator menangani distribusi gradient calculation di perangkat yang sesuai
        accelerator.backward(loss)
        # Melakukan optimasi berdasarkan gradient
        optimizer.step()
        # Melakukan update terhadap learning rate
        lr_scheduler.step()
        # Mengatur gradient menjadi nol
        optimizer.zero_grad()
        # Mengupdate progress bar
        progress_bar.update(1)
